# Dogs vs. Cats Redux: Kernels Edition

## 2. 모델구성 및 학습

In [1]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import time
import re

# 구현모듈
import data
import model
import train

/home/shkim/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### 2.1 학습 데이터 생성 함수

In [2]:
def getFiles(dir, word):
    files =   [dir+i for i in os.listdir(dir) if word in i]
    files.sort(key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
    return files

In [3]:
def getDogCatData(image_dir, test_size=0.3):
    dog_files =  getFiles(image_dir, 'dog')
    cat_files =  getFiles(image_dir, 'cat')
    print("dog_files: ", len(dog_files))
    print("cat_files: ", len(cat_files))
    
    #dog_files = dog_files[:200]
    #cat_files = cat_files[:200]

    dog_cat_files = dog_files + cat_files

    label_dog = [0 for i in range(len(dog_files))]
    label_cat = [1 for i in range(len(cat_files))]
    label_one_hot = np.eye(2)[label_dog + label_cat]

    return train_test_split (dog_cat_files, label_one_hot, test_size=test_size, random_state=42)

### 2.2 학습데이터와 valid 데이터 분리

In [4]:
DATA_DIR = "../data/"
TRAIN_DIR = DATA_DIR + "train_resize/"

x_train, x_valid, y_train, y_valid = getDogCatData (TRAIN_DIR)

print("train: ", len(x_train))
print("valid: ", len(x_valid))

dog_files:  12500
cat_files:  12500
train:  17500
valid:  7500


In [5]:
learning_rate = 0.0001
epochs = 50
batch_size =8

### 2.3 dataset 생성

In [6]:
# dataset 생성
train_dataset = data.Dataset_image([x_train, y_train], batch_size = batch_size)
valid_dataset = data.Dataset_image([x_valid, y_valid], batch_size = batch_size)

In [7]:
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True)))
model = model.Inception_slim(sess, "model")
model.build_net(image_shape=[150, 150, 3], class_count=2)

Instructions for updating:
Use the retry module or similar alternatives.
Tensor("model/stem/stem/concat:0", shape=(?, 150, 150, 96), dtype=float32)
Tensor("model/inception-A/inception_block_a0/concat:0", shape=(?, 150, 150, 96), dtype=float32)
Tensor("model/inception-A/inception_block_a1/concat:0", shape=(?, 150, 150, 96), dtype=float32)
Tensor("model/reduction-A/reduction_A/concat:0", shape=(?, 75, 75, 256), dtype=float32)
Tensor("model/inception-B/inception_block_b0/concat:0", shape=(?, 75, 75, 256), dtype=float32)
Tensor("model/inception-B/inception_block_b1/concat:0", shape=(?, 75, 75, 256), dtype=float32)
Tensor("model/inception-B/inception_block_b2/concat:0", shape=(?, 75, 75, 256), dtype=float32)
Tensor("model/reduction-B/reduction_B/concat:0", shape=(?, 38, 38, 384), dtype=float32)
Tensor("model/inception-C/inception_block_c0/concat:0", shape=(?, 38, 38, 384), dtype=float32)


### 2.4 학습

In [8]:
train.train (model, train_dataset, valid_dataset, learning_rate, epochs)

train_count:  17500 total_batch_train:  2188
valid_count:  7500 total_batch_valid:  938
Learning started. It takes sometime.


ResourceExhaustedError: OOM when allocating tensor with shape[8,32,150,150] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: model/stem/stem/conv2d/Conv2D = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](model/stem/stem/conv2d/Conv2D-0-TransposeNHWCToNCHW-LayoutOptimizer, model/stem/stem/conv2d/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: optimizer/update/_296 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_57733_optimizer/update", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want 

### 2.5 평가

In [ ]:
train.eval (model, train_dataset)

In [ ]:
train.eval (model, valid_dataset)